In [2]:
import cv2
print(cv2.__version__)

4.5.3


# 图像载入与显示

In [3]:
# 读取图像
img1 = cv2.imread("E:/jupyter_notebook_files/ImageAndVideoProcess/test1/1.jpg")
# 创建一个名为lesson的窗口
cv2.namedWindow("lesson")
# 显示图像
cv2.imshow("lesson", img1)
# hold image
cv2.waitKey(0)
cv2.destroyAllWindows()

# 将原图的RGB三通道分离并显示

In [4]:
b,g,r = cv2.split(img1)
cv2.imshow("exampleB", b)  # 展示B通道图
cv2.imshow("exampleG", g)
cv2.imshow("exampleR", r)
#hold image
cv2.waitKey(0)
cv2.destroyAllWindows()

# 将原图像进行直方图均衡化并将处理后的结果保存至equalizeHistImage中显示并保存

## 对曝光过度或者逆光拍摄的图片可以通过直方图均衡化的方法用来增强局部或者整体的对比度

![jupyter](https://docs.opencv.org/3.4.0/histogram_sample.jpg)

##### 直方图的左边部分显示了图像中较暗像素的数量，右边区域显示了更明亮的像素。从直方图中可以看到，深色区域的像素数量比亮色区域更多，而中间色调的数量(中值大约在127左右)则少得多。

### 绘制img1的直方图 

In [5]:
from matplotlib import pyplot as plt

plt.hist(img1.ravel(), 256, [0,256])
plt.show()

<Figure size 640x480 with 1 Axes>

#### 直方图均衡化
- ##### 考虑一个图像，其像素值仅限制在特定的值范围内。例如，更明亮的图像将使所有像素都限制在高值中。但是一个好的图像会有来自图像的所有区域的像素。所以你需要把这个直方图拉伸到两端(如下图所给出的)，这就是直方图均衡的作用(用简单的话说)。这通常会改善图像的对比度

![](https://docs.opencv.org/3.4.0/histogram_equalization.png)

###### OpenCV有一个函数可以这样做，cv.equalizeHist()，它封装好了计算cdf和cdf重映射以及根据cdf表生成直方图均衡图像的过程。它的输入只是灰度图像，输出是我们的直方图均衡图像。

In [6]:
import numpy as np 

cv2.imshow("before", img1)

# Histogram Equalization
r2 = cv2.equalizeHist(r)
g2 = cv2.equalizeHist(g)
b2 = cv2.equalizeHist(b)

equalizeHistImage = img1.copy()
equalizeHistImage[:,:,0] = g2
equalizeHistImage[:,:,1] = b2
equalizeHistImage[:,:,2] = r2

#print(equalizeHistImage)

cv2.imshow("after", equalizeHistImage)
#plt.show()
cv2.waitKey(0)

-1

# 将原图像进行锐化处理并将处理后的结果保存至filter2DImage中并显示

In [8]:
blur_img = cv2.GaussianBlur(img1, (0, 0), 5)
usm = cv2.addWeighted(img1, 1.5, blur_img, -0.5, 0)
#cv.addWeighted(图1,权重1, 图2, 权重2, gamma修正系数, dst可选参数, dtype可选参数)
cv2.imshow("mask image", usm)

#锐化处理
h, w = img1.shape[:2]
result = np.zeros([h, w*2, 3], dtype=img1.dtype)
result[0:h,0:w,:] = img1
result[0:h,w:2*w,:] = usm
cv2.putText(result, "original image", (10, 30), cv2.FONT_ITALIC, 1.0, (0, 0, 255), 2)
cv2.putText(result, "sharpen image", (w+10, 30), cv2.FONT_ITALIC, 1.0, (0, 0, 255), 2)
#cv2.putText(图像名，标题，（x坐标，y坐标），字体，字的大小，颜色，字的粗细）
cv2.imshow("sharpen_image", result)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 将源图像进行伽马矫正，利用滑动条对参数进行调整

- ##### 用来图像增强，其提升了暗部细节，简单来说就是通过非线性变换，让图像从暴光强度的线性响应变得更接近人眼感受的响应，即将漂白（相机曝光）或过暗（曝光不足）的图片，进行矫正
- 伽马值小于1时，会拉伸图像中灰度级较低的区域，同时会压缩灰度级较高的部分
- 伽马值大于1时，会拉伸图像中灰度级较高的区域，同时会压缩灰度级较低的部分

In [20]:
import copy

#伽马变换
gamma=copy.deepcopy(img1)
rows=img1.shape[0]
cols=img1.shape[1]
for i in range(rows):
    for j in range(cols):
        gamma[i][j]=20*pow(gamma[i][j],0.8)

#通过窗口展示图片 第一个参数为窗口名 第二个为读取的图片变量
cv2.imshow('original',img1)
cv2.imshow('gamma',gamma)

#暂停cv2模块 不然图片窗口一瞬间即就会消失 观察不到
cv2.waitKey(0)

-1

### 滑动条回调 

In [1]:
import copy
import cv2
import numpy as np
#导入图像
img = cv2.imread("E:/jupyter_notebook_files/ImageAndVideoProcess/test1/1.jpg")
#新建一个窗口
cv2.namedWindow('gamma')
#定义回调函数，这里pass表示不做任何事，用作占位语句保持程序结构的完整性
def nothing(x):
    pass
#新建一个滑动条
cv2.createTrackbar('intensity','gamma',1, 500, nothing)

intensity = 250
while(1):
    gamma = np.power(img/float(np.max(img)), intensity*0.01)
    cv2.imshow('gamma',gamma)
    #返回滑块所在位置对应的值
    intensity = cv2.getTrackbarPos('intensity','gamma')
    if cv2.waitKey(1)==ord('q'):
        break
cv2.destroyAllWindows()